In [1]:
# %pip install torch
# %pip install pymoo
# %pip install pywin32

In [2]:
import os, sys

# Get the absolute path of the notebook's directory
notebook_dir = os.getcwd()

# Navigate to the parent directory
parent_dir = os.path.abspath(os.path.join(notebook_dir, ".."))

# Add the parent directory to sys.path so we can import modules
sys.path.append(parent_dir)

# Verify the path
print(f"Added to sys.path: {parent_dir}")

# Add it to sys.path
sys.path.append(parent_dir)

print(parent_dir)
from localityaware.module import *
from NSGA_nn.nsga import *
#from FlashOperation.Refrig2DrumHeatExConstr1 import Refrig2DrumConstraintHeatExConstrforNN, Refrig2Drumproblem

Added to sys.path: /Users/panwapromtep/Library/CloudStorage/OneDrive-JohnsHopkins/Process Design/Aspen
/Users/panwapromtep/Library/CloudStorage/OneDrive-JohnsHopkins/Process Design/Aspen
['/Users/panwapromtep/miniforge3/envs/torch/lib/python312.zip', '/Users/panwapromtep/miniforge3/envs/torch/lib/python3.12', '/Users/panwapromtep/miniforge3/envs/torch/lib/python3.12/lib-dynload', '', '/Users/panwapromtep/miniforge3/envs/torch/lib/python3.12/site-packages', '/Users/panwapromtep/Library/CloudStorage/OneDrive-JohnsHopkins/Process Design/Aspen', '/Users/panwapromtep/Library/CloudStorage/OneDrive-JohnsHopkins/Process Design/Aspen', '/Users/panwapromtep/Library/CloudStorage/OneDrive-JohnsHopkins/Process Design/Aspen']


In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import torch
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import pickle
import os
import time

# Define the file path for saving/loading the data
data_file = "flash_simulation_data_sim_new.pkl"



print("Generating new data...")
assSim = Refrig2DrumConstraintHeatExConstrforNN(AspenFile="../FlashOperation/FlashOperation_HeatExchanger.bkp",
                                        wdpath="../FlashOperation",
                                        visibility=False
                                        )

In [4]:
import os
import time
import pickle

CACHE_FILE = "../FlashOperation/results/flash/data_sim_cache.pkl"
print(os.path.exists(CACHE_FILE))
# Try to load from pickle cache
if os.path.exists(CACHE_FILE):
    with open(CACHE_FILE, "rb") as f:
        cache = pickle.load(f)
    data_sim                    = cache["data"]
    data_gen_time               = cache["gen_time"]
    total_original_assSim_calls = cache["assSim_calls"]
    flash_1_range               = cache["flash_1_range"]
    flash_2_range               = cache["flash_2_range"]
    print(f"Loaded cache (shape={data_sim.shape}, sim calls={total_original_assSim_calls}, gen time={data_gen_time:.2f}s)")
else:
    # Define parameter ranges
    flash_1_range = np.linspace(1, 20, 7)
    flash_2_range = np.linspace(1, 20, 7)

    start_time = time.time()
    data_sim = []
    for flash_1 in flash_1_range:
        for flash_2 in flash_2_range:
            print(flash_1, flash_2)
            x_unflat = assSim.unflatten_params([float(flash_1), float(flash_2)])
            f, g = assSim.run_obj(x_unflat)
            data_sim.append([flash_1, flash_2, f, g])

    data_sim = np.array(data_sim, dtype=float)
    data_gen_time               = time.time() - start_time
    total_original_assSim_calls = len(data_sim)

    # Build cache dict
    cache = {
        "data": data_sim,
        "gen_time": data_gen_time,
        "assSim_calls": total_original_assSim_calls,
        "flash_1_range": flash_1_range,
        "flash_2_range": flash_2_range,
    }
    # Write to pickle
    with open(CACHE_FILE, "wb") as f:
        pickle.dump(cache, f)
    print(f"Simulation done in {data_gen_time:.2f}s → saved to {CACHE_FILE}")

print("data shape:", data_sim.shape)
print("sim time (s):", data_gen_time)
print("assSim calls:", total_original_assSim_calls)


True
Loaded cache (shape=(49, 4), sim calls=49, gen time=20.15s)
data shape: (49, 4)
sim time (s): 20.154319047927856
assSim calls: 49


In [ ]:
import numpy as np
import torch

# ---- Inspect original data ----
print("Number of samples:", len(data_sim))
data_sim = data_sim.astype(float)
print("dtype:", data_sim.dtype)
print("first 5 rows:\n", data_sim[:5])

# ---- 1) Replace NaNs in the fourth column with 30 ----
#    (column index 3 since Python is zero‑based)
nan_mask = np.isnan(data_sim[:, 3])
if np.any(nan_mask):
    print(f"Found {nan_mask.sum()} NaNs in column 3 – replacing with 30.0")
    data_sim[nan_mask, 3] = 30.0

# ---- 2) Scale inputs & outputs correctly ----
scaler = TorchMinMaxScaler(
    feature_range=(-1, 1), 
    min_vals=[1, 1],
    max_vals=[20, 20],
    scale_y=True,
    min_y=[1000, -100],
    max_y=[4e5, 30]
)

# Prepare tensors
X_raw = torch.tensor(data_sim[:, :2], dtype=torch.float32)
Y_raw = torch.tensor(data_sim[:, 2:], dtype=torch.float32)

# Transform
X_scaled_t, Y_scaled_t = scaler.transform(X_raw, Y_raw)

# Convert back to NumPy for your dataset
X_scaled = X_scaled_t.detach().cpu().numpy()
Y_scaled = Y_scaled_t.detach().cpu().numpy()

# Combine into one array
data_sim_scaled = np.hstack([X_scaled, Y_scaled])
print("Scaled data shape:", data_sim_scaled.shape)
print("First 5 rows scaled:\n", data_sim_scaled[:5])

# ---- Initialize your dataset & model ----
dataset = DynamicDataset(data_sim_scaled, num_inputs=2)
model   = MLP(2, [20, 20, 20], 2)

print("✅ Dataset and model ready. No NaNs remain, and scaling is applied correctly.")


Number of samples: 49
dtype: float64
first 5 rows:
 [[ 1.00000000e+00  1.00000000e+00  2.30606519e+04 -6.50630049e+01]
 [ 1.00000000e+00  4.16666667e+00  3.84869866e+04 -6.54195554e+01]
 [ 1.00000000e+00  7.33333333e+00  5.67770989e+04 -6.35082882e+01]
 [ 1.00000000e+00  1.05000000e+01  6.19077758e+04 -6.58798164e+01]
 [ 1.00000000e+00  1.36666667e+01  6.64762071e+04 -6.62571315e+01]]
Found 3 NaNs in column 3 – replacing with 30.0
Scaled data shape: (49, 4)
First 5 rows scaled:
 [[-1.         -1.         -0.8894203  -0.46250772]
 [-1.         -0.6666667  -0.8120953  -0.46799314]
 [-1.         -0.3333333  -0.7204156  -0.4385891 ]
 [-1.          0.         -0.69469786 -0.47507405]
 [-1.          0.33333337 -0.67179847 -0.48087895]]
✅ Dataset and model ready. No NaNs remain, and scaling is applied correctly.


In [ ]:
problem = Refrig2Drumproblem(model, scaler)
def my_ga_factory(it, prev_pop):
    if prev_pop is None:
        return GA(
            pop_size=1000,
            sampling=LHS(),
            eliminate_duplicates=False
        )
    else:
        return GA(
            pop_size=1000,
            sampling=ResumeFromPopulation(prev_pop),
            eliminate_duplicates=False
        )

In [12]:
out = optimize_surr_ga(model=model,
                   dataset=dataset,
                   assSim=assSim,
                   problem=problem,
                   algo_factory=my_ga_factory,
                   lrs={'first':1e-4, 'others':1e-4},
                   epochs={'first':3000, 'others':500},
                   scaler=scaler,
                   device='cpu',
                   iter=10,
                   print_loss=True,
                   print_it_data=True,
                   pop_size=1000,
                   n_gen = 3,
                   new_data_size=15,
                   batch_size=128
                   )

Iteration 0: Training surrogate model...
data_point: [tensor([[ 0.0000, -1.0000],
        [-0.6667,  0.6667],
        [-1.0000,  0.0000],
        [ 1.0000,  0.0000],
        [ 0.3333, -1.0000],
        [-0.3333,  0.6667],
        [ 1.0000, -1.0000],
        [ 0.0000,  1.0000],
        [-0.6667, -1.0000],
        [ 0.0000,  0.3333],
        [ 0.0000, -0.3333],
        [ 0.6667, -0.3333],
        [-1.0000, -0.3333],
        [ 0.3333,  1.0000],
        [ 0.0000,  0.0000],
        [ 1.0000, -0.3333],
        [-0.3333,  0.3333],
        [-0.3333,  0.0000],
        [ 0.6667,  0.6667],
        [-0.6667, -0.3333],
        [ 0.6667,  1.0000],
        [-0.6667, -0.6667],
        [ 0.6667, -1.0000],
        [ 0.0000,  0.6667],
        [-1.0000,  0.6667],
        [ 0.3333, -0.3333],
        [ 0.3333,  0.3333],
        [-0.6667,  0.3333],
        [ 0.6667,  0.3333],
        [-0.3333, -1.0000],
        [-0.3333, -0.6667],
        [ 0.3333, -0.6667],
        [ 0.3333,  0.0000],
        [-0.3333, -0.3

AttributeError: 'str' object has no attribute 'bounds'

In [ ]:
import os
import datetime
import pickle

print(os.getcwd())
# Get current notebook name manually (or hardcode it here)
notebook_name = "flash"  # or os.path.basename(__file__).replace(".py", "")

# Get timestamp
now = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

# Construct path: results/<notebook_name>/run_<timestamp>/
base_dir = os.path.join("../NSGA_nn/results", notebook_name)
save_dir = os.path.join(base_dir, f"run_{now}")
os.makedirs(save_dir, exist_ok=True)
print(save_dir)

print(f"Results will be saved in: {save_dir}")

# Save out.pkl
out["data_gen_time"] = data_gen_time
out["total_original_assSim_calls"] = total_original_assSim_calls
with open(os.path.join(save_dir, "out.pkl"), "wb") as f:
    pickle.dump(out, f)

# Save scaler
with open(os.path.join(save_dir, "scaler.pkl"), "wb") as f:
    pickle.dump(scaler, f)

# Save dataset
with open(os.path.join(save_dir, "dataset.pkl"), "wb") as f:
    pickle.dump(dataset, f)

print(f"Saved all results in {save_dir}")
